In [22]:
!pip install findspark

In [2]:
!pip install Pandas

  Using cached pandas-2.2.2-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.2-cp311-cp311-win_amd64.whl (11.6 MB)
Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl (15.8 MB)
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)


In [23]:
import findspark

findspark.init()

from pyspark.sql import SparkSession

scala_version = '2.12'  # your scala version
spark_version = '3.5.0'  # your spark version
packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:3.6.0'  #your kafka version
]
spark = SparkSession.builder.master("local").appName("kafka-example").config("spark.jars.packages",
                                                                             ",".join(packages)).getOrCreate()
spark

In [24]:
topic_name = 'RandomNumber'
kafka_server = 'localhost:29092'
kafkaDf = spark.read.format("kafka").option("kafka.bootstrap.servers", kafka_server).option("subscribe",
                                                                                            topic_name).option(
    "startingOffsets",
    "earliest").load()


In [25]:
kafkaDf.toPandas()

,key,value,topic,partition,offset,timestamp,timestampType
0,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,0,2024-05-21 11:48:21.886,0
1,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,1,2024-05-21 11:48:22.886,0
2,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,2,2024-05-21 11:48:23.887,0
3,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,3,2024-05-21 11:48:24.889,0
4,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,4,2024-05-21 11:53:13.236,0
...,...,...,...,...,...,...,...
1049,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,1049,2024-05-21 12:21:27.504,0
1050,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,1050,2024-05-21 12:21:28.507,0
1051,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,1051,2024-05-21 12:21:29.508,0
1052,None,"[123, 34, 110, 117, 109, 98, 101, 114, 34, 58,...",RandomNumber,0,1052,2024-05-21 12:21:30.515,0


In [26]:
from pyspark.sql.functions import col

batchDF = kafkaDf.select(col('topic'), col('offset'), col('value').cast('string').substr(12, 1).alias('rand_number'))
from time import sleep
from IPython.display import display, clear_output

for x in range(0, 2000):
    try:
        print("Showing live view refreshed every 5 seconds")
        print(f"Seconds passed: {x * 5}")
        display(batchDF.toPandas())
        sleep(5)
        clear_output(wait=True)
    except KeyboardInterrupt:
        print("break")
    break
print("Live view ended...")

Live view ended...


In [27]:
batchCountDF = batchDF.groupBy('rand_number').count()
for x in range(0, 2000):
    try:
        print("Showing live view refreshed every 5 seconds")
        print(f"Seconds passed: {x * 5}")
        display(batchCountDF.toPandas())
        sleep(100)
        clear_output(wait=True)
    except KeyboardInterrupt:
        print("break")
    break
print("Live view ended...")

Live view ended...
